In [54]:
import pandas as pd
import re
import time
import itertools
import pinyin
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
df = pd.read_csv('hsk_cleaned.csv')

In [3]:
df_hsk1 = df[df['Level'] == '1']
df_hsk2 = df[df['Level'] == '2']
df_hsk3 = df[df['Level'] == '3']
df_hsk4 = df[df['Level'] == '4']
df_hsk5 = df[df['Level'] == '5']
df_hsk6 = df[df['Level'] == '6']
df_hsk7_9 = df[df['Level'] == '7-9']

In [4]:
hsk1_hanzi_set = set(list(''.join([word for word in (df_hsk1['Simplified'] + df_hsk1['Traditional'])])))
hsk2_hanzi_set = set(list(''.join([word for word in (df_hsk2['Simplified'] + df_hsk2['Traditional'])])))
hsk3_hanzi_set = set(list(''.join([word for word in (df_hsk3['Simplified'] + df_hsk3['Traditional'])])))
hsk4_hanzi_set = set(list(''.join([word for word in (df_hsk4['Simplified'] + df_hsk4['Traditional'])])))
hsk5_hanzi_set = set(list(''.join([word for word in (df_hsk5['Simplified'] + df_hsk5['Traditional'])])))
hsk6_hanzi_set = set(list(''.join([word for word in (df_hsk6['Simplified'] + df_hsk6['Traditional'])])))
hsk7_9_hanzi_set = set(list(''.join([word for word in (df_hsk7_9['Simplified'] + df_hsk7_9['Traditional'])])))
hsk_hanzi_set = set(list(''.join([word for word in (df['Simplified'] + df['Traditional'])])))

In [5]:
hsk1_word_set = set(list(df_hsk1['Simplified']) + list(df_hsk1['Traditional']))
hsk2_word_set = set(list(df_hsk2['Simplified']) + list(df_hsk2['Traditional']))
hsk3_word_set = set(list(df_hsk3['Simplified']) + list(df_hsk3['Traditional']))
hsk4_word_set = set(list(df_hsk4['Simplified']) + list(df_hsk4['Traditional']))
hsk5_word_set = set(list(df_hsk5['Simplified']) + list(df_hsk5['Traditional']))
hsk6_word_set = set(list(df_hsk6['Simplified']) + list(df_hsk6['Traditional']))
hsk7_9_word_set = set(list(df_hsk7_9['Simplified']) + list(df_hsk7_9['Traditional']))
hsk_word_set = set(list(df['Simplified']) + list(df['Traditional']))

In [7]:
def get_word_pinyin(word):
    if word in word_pinyin_map.keys()


# for a given hanzi, return the lowest level that it appears in
def get_hanzi_level(hanzi):
    if hanzi in hsk1_hanzi_set:
        return '1'
    if hanzi in hsk2_hanzi_set:
        return '2'
    if hanzi in hsk3_hanzi_set:
        return '3'
    if hanzi in hsk4_hanzi_set:
        return '4'
    if hanzi in hsk5_hanzi_set:
        return '5'
    if hanzi in hsk6_hanzi_set:
        return '6'
    if hanzi in hsk7_9_hanzi_set:
        return '7-9'
    return 'Unknown'

In [8]:
# Create a dictionary of characters where each character

In [81]:
def is_hanzi(char):
    # hanzi are between U+4E00 and U+9FFF
    return ord(char) <= 40959 and ord(char) >= 19968


def is_all_hanzi(string):
    return all([is_hanzi(char) for char in list(string)])

# this function was written by ChatGPT
def escape_regex(string):
    regex_reserved_chars = r'\^$.|?*+(){}[]'
    escaped_string = re.sub(f'([{re.escape(regex_reserved_chars)}])', r'\\\1', string)
    return escaped_string


# split text into chunks of Hanzi split by non-Hanzi characters
def split_into_hanzi_chunks(text):
    if is_all_hanzi(text):
        return [text]
    
    chunks = []
    
    if not isinstance(text, str):
        return []
    
    non_hanzi_characters = []
    
    for char in list(text):
        if not is_hanzi(char):
            non_hanzi_characters.append(escape_regex(char))
    
    regex_split_string = '|'.join(list(set(non_hanzi_characters)))
    split_string = re.split(regex_split_string, text)
    
    for string in split_string:
        if len(string) > 0:
            chunks.append(string)
            
    return chunks


# reverse of the function above
def split_into_non_hanzi_chunks(text):
    chunks = []
    
    if not isinstance(text, str):
        return []
    
    curr_chunk = ''
    
    for char in list(text):
        if not is_hanzi(char):
            curr_chunk += char
        if is_hanzi(char) and curr_chunk != '':
            chunks.append(curr_chunk)
            curr_chunk = ''
            
    return chunks

    
def get_hsk_word_level(word):
    if len(word) == 1:
        return get_hanzi_level(word)
    if word in hsk1_word_set:
        return '1'
    elif word in hsk2_word_set:
        return '2'
    elif word in hsk3_word_set:
        return '3'
    elif word in hsk4_word_set:
        return '4'
    elif word in hsk5_word_set:
        return '5'
    elif word in hsk6_word_set:
        return '6'
    elif word in hsk7_9_word_set:
        return '7-9'
    return 'Unknown'


def word_in_hsk(word):
    return word in hsk_word_set or word in hsk_hanzi_set

'''
# '法庭将于明年开庭审理他们涉嫌为中国从事间谍活动的案子'
# sliding window parsing algorithm
def tokenize_hanzi_string(hanzi_text):
    tokens = []
    text_length = len(hanzi_text)
    curr_index = 1
    
    while hanzi_text != '':
        curr_word = hanzi_text[:curr_index]
        if not word_in_hsk(curr_word):
            if curr_index > 1:
                new_word = hanzi_text[:curr_index - 1]
                hanzi_text = hanzi_text[curr_index - 1:]
            else:
                new_word = hanzi_text[:curr_index]
                hanzi_text = hanzi_text[curr_index:]
            new_word_level = get_hsk_word_level(new_word)
            new_token = [new_word, new_word_level]
            tokens.append(new_token)
            curr_index = 1
        else:
            # if this is the last word to parse
            if curr_word == hanzi_text:
                new_word = hanzi_text[:curr_index]
                hanzi_text = ''
                new_word_level = get_hsk_word_level(new_word)
                new_token = [new_word, new_word_level]
                tokens.append(new_token)
            curr_index += 1
            
    return tokens
'''
# for a string s, returns the length of the longest of s[:1], s[:2], s[:3], s[:4] that is an hsk word
# if none of those prefix substrings are in the hsk word set, then it returns the shortest substring (s[:1])

# in simpler terms, given a string, what is the prefix substring of the string that should be considered the next token
def get_next_token_length(string):
    next_possible_tokens = []
    
    for i in range(1, min([len(string) + 1, 5])):
        next_possible_tokens.append(string[:i])
        
    while len(next_possible_tokens) > 0:
        popped_token = next_possible_tokens.pop()
        if get_hsk_word_level(popped_token) != 'Unknown':
            return len(popped_token)
        
    return 1


def tokenize_hanzi_string(string):
    tokens = []
    
    while len(string) > 0:
        next_index = get_next_token_length(string)
        new_word = string[:next_index]
        new_word_level = get_hsk_word_level(new_word)
        new_word_pinyin = pinyin.get(new_word)
        new_token = [new_word, new_word_level, new_word_pinyin]
        tokens.append(new_token)
        string = string[next_index:]
        print(string)
    return tokens


def parse_text(text):
    tokens = []
    hanzi_chunks = split_into_hanzi_chunks(text)
    non_hanzi_chunks = split_into_non_hanzi_chunks(text)
    
    if len(hanzi_chunks) == 0:
        chunks = non_hanzi_chunks
    elif len(non_hanzi_chunks) == 0:
        chunks = hanzi_chunks
    # interweave the two chunks depending on which type of chunk (hanzi or non-hanzi)
    # appears in the text
    elif is_hanzi(text[0]):
        chunks = [chunk for chunk in itertools.chain(*itertools.zip_longest(hanzi_chunks, non_hanzi_chunks)) if chunk is not None]
    else:
        chunks = [chunk for chunk in itertools.chain(*itertools.zip_longest(non_hanzi_chunks, hanzi_chunks)) if chunk is not None]
    
    print(chunks)
        
    
    for chunk in chunks:
        # is this a hanzi chunk? If so, get tokens for it and append them to overall token list
        if is_hanzi(chunk[0]):
            chunk_tokens = tokenize_hanzi_string(chunk)
            tokens.extend(chunk_tokens)
        else:
            non_hanzi_token = [chunk, 'Not Hanzi']
            tokens.append(non_hanzi_token)
            
    return tokens

In [32]:
get_next_possible_tokens('为什么我的函数有这么大的问题')

3

In [83]:
t1 = time.time()
parsed_text = parse_text('''
华盛顿 — 
中国在台湾总统赖清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测。美国多位军方官员此前对习近平攻台的时间表做出了种种预测。美国应该如何限制中国获得包括尖端芯片在内的关键技术，来阻遏北京武力攻台呢？前美国官员和技术专家认为，美国应该从现在就开始，采取一切必要行动确保中国对台湾的军事冒险不会得逞，包括推动多位盟友参与的复边出口控制体系，以及把现有的出口控制扩大到所有的芯片制造设备，使中国无法生产芯片。
''')
t2 = time.time()
print(t2 - t1)

['\n', '华盛顿', ' — \n', '中国在台湾总统赖清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测', '。', '美国多位军方官员此前对习近平攻台的时间表做出了种种预测', '。', '美国应该如何限制中国获得包括尖端芯片在内的关键技术', '，', '来阻遏北京武力攻台呢', '？', '前美国官员和技术专家认为', '，', '美国应该从现在就开始', '，', '采取一切必要行动确保中国对台湾的军事冒险不会得逞', '，', '包括推动多位盟友参与的复边出口控制体系', '，', '以及把现有的出口控制扩大到所有的芯片制造设备', '，', '使中国无法生产芯片']
盛顿
顿

在台湾总统赖清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
台湾总统赖清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
湾总统赖清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
总统赖清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
赖清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
清德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
德就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
就任后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
后举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
举行环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
环台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
台军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
军演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
演的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
的举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
举动再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
再次引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
引发台海局势的紧张以及北京是否要武力夺取台湾的猜测
台海局

In [84]:
parsed_text

[['\n', 'Not Hanzi'],
 ['华', '3', 'huá'],
 ['盛', '6', 'shèng'],
 ['顿', '3', 'dùn'],
 [' — \n', 'Not Hanzi'],
 ['中国', '1', 'zhōngguó'],
 ['在', '1', 'zài'],
 ['台', '3', 'tái'],
 ['湾', '6', 'wān'],
 ['总统', '4', 'zǒngtǒng'],
 ['赖', '6', 'lài'],
 ['清', '2', 'qīng'],
 ['德', '5', 'dé'],
 ['就任', '7-9', 'jìurèn'],
 ['后', '1', 'hòu'],
 ['举行', '2', 'jǔxíng'],
 ['环', '3', 'huán'],
 ['台', '3', 'tái'],
 ['军', '5', 'jūn'],
 ['演', '3', 'yǎn'],
 ['的', '1', 'de'],
 ['举动', '5', 'jǔdòng'],
 ['再次', '5', 'zàicì'],
 ['引发', '7-9', 'yǐnfā'],
 ['台', '3', 'tái'],
 ['海', '2', 'hǎi'],
 ['局势', '7-9', 'júshì'],
 ['的', '1', 'de'],
 ['紧张', '3', 'jǐnzhāng'],
 ['以及', '4', 'yǐjí'],
 ['北京', '1', 'běijīng'],
 ['是否', '4', 'shìfǒu'],
 ['要', '1', 'yào'],
 ['武力', '7-9', 'wǔlì'],
 ['夺取', '6', 'duóqǔ'],
 ['台', '3', 'tái'],
 ['湾', '6', 'wān'],
 ['的', '1', 'de'],
 ['猜测', '5', 'cāicè'],
 ['。', 'Not Hanzi'],
 ['美', '3', 'měi'],
 ['国', '1', 'guó'],
 ['多', '1', 'duō'],
 ['位', '2', 'wèi'],
 ['军', '5', 'jūn'],
 ['方', '1', 'fāng'],
 ['官员

In [74]:
get_next_token_length('芯片')

1

In [88]:
'迫不得已' in hsk_word_set

False